In [ ]:
MAX_LEN = 512 
llama_checkpoint = "meta-llama/Llama-2-7b-hf"

In [ ]:
from datasets import load_dataset
dataset = load_dataset("mehdiiraqui/twitter_disaster")

In [ ]:
dataset

In [ ]:
from datasets import Dataset
# Split the dataset into training and validation datasets
data = dataset['train'].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
data['val'] = data.pop("test")
# Convert the test dataframe to HuggingFace dataset and add it into the first dataset
data['test'] = dataset['test']

In [ ]:
data

In [ ]:
import pandas as pd

data['train'].to_pandas().info()
data['test'].to_pandas().info()

In [ ]:
pos_weights = len(data['train'].to_pandas()) / (2 * data['train'].to_pandas().target.value_counts()[1])
neg_weights = len(data['train'].to_pandas()) / (2 * data['train'].to_pandas().target.value_counts()[0])

In [ ]:
POS_WEIGHT, NEG_WEIGHT = (pos_weights, neg_weights)
POS_WEIGHT, NEG_WEIGHT

In [ ]:
# Number of Characters
max_char = data['train'].to_pandas()['text'].str.len().max()
# Number of Words
max_words = data['train'].to_pandas()['text'].str.split().str.len().max()
max_char, max_words


In [ ]:
data['train'][0]

In [ ]:
# Load Llama 2 Tokenizer
col_to_delete = ['id', 'keyword','location', 'text']

from transformers import AutoTokenizer, DataCollatorWithPadding
llama_tokenizer = AutoTokenizer.from_pretrained(llama_checkpoint, add_prefix_space=True)
llama_tokenizer.pad_token_id = llama_tokenizer.eos_token_id
llama_tokenizer.pad_token = llama_tokenizer.eos_token

def llama_preprocessing_function(examples):
    return llama_tokenizer(examples['text'], truncation=True, max_length=MAX_LEN)

llama_tokenized_datasets = data.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)
llama_tokenized_datasets = llama_tokenized_datasets.rename_column("target", "label")
llama_tokenized_datasets.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
llama_data_collator = DataCollatorWithPadding(tokenizer=llama_tokenizer)


In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
llama_checkpoint = "meta-llama/Llama-2-7b-hf"
llama_model =  AutoModelForSequenceClassification.from_pretrained(
  pretrained_model_name_or_path=llama_checkpoint,
  num_labels=2,
  device_map="auto",
  offload_folder="offload",
  trust_remote_code=True
)


In [ ]:
llama_model.config.pad_token_id = llama_model.config.eos_token_id


In [1]:
from transformers import AutoModel

model_name = "meta-llama/Llama-2-7b-chat-hf"  # Replace with your model name
model = AutoModel.from_pretrained(model_name)

model.save_pretrained('./model')  # Saves the model to a local directory


/WAVE/projects/newsq_scu/xiaoxiao_git/Llama-race/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 614/614 [00:00<00:00, 1.95MB/s]
model.safetensors.index.json: 100%|██████████| 26.8k/26.8k [00:00<00:00, 30.7MB/s]
  warnings.warn(
/WAVE/projects/newsq_scu/xiaoxiao_git/Llama-race/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 9976.58 MB. The target location /tmp/xdgcache-xshang/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf/blobs only has 1.22 MB free disk space.
  warnings.warn(
model-00001-of-00002.safetensors:   0%|          | 10.5M/9.98G [00:00<01:27, 114MB/s]


OSError: [Errno 28] No space left on device